# Topic ideas

---

Student name: Priscila Strömsdörfer - ps178@hdm-stuttgart.de

---

## Topic idea 1: Regressionsbasierte Optimierung kritischer Material-Reichweiten in Logistiksystemen von Mercedes-Benz: AKL, PCC und Anlauflager

### Data source

*Der Datensatz stammt aus den internen SAP-basierten Logistiksystemen der Mercedes-Benz AG. Er umfasst Materialverfügbarkeitsdaten aus dem Automatischen Kleinteillager (AKL), den Pick-and-Pack-Zentren (PCC) sowie den Anlauflagern. Die Daten werden aus verschiedenen SAP-Tabellen wie MARD, MARC, EKET, BEDZL und weiteren abgerufen. Sie bilden eine vollständige Grundlage zur Analyse von Beständen, Bedarfen und Bestellungen entlang der Supply Chain.*

### Data characteristics

| **Kategorie**              | **Datenquelle (SAP-Tabellen)** | **Merkmale**                                   | **Beschreibung**                                                                 |
|----------------------------|--------------------------------|-----------------------------------------------|---------------------------------------------------------------------------------|
| **Lagerdaten**             | MARD                          | - `LABST`: Verfügbarer Bestand                | Verfügbarer Bestand für jedes Material, Werk und Lagerort.                      |
|                            |                                | - Sicherheitsbestand                          | Mindestbestand, der gehalten werden sollte, um Engpässe zu vermeiden.           |
|                            |                                | - Kapazitätsdaten                             | Maximal verfügbare Lagerkapazität.                                              |
| **Materialstammdaten**     | MARC, MAKT                    | - `MATKL`: Materialgruppe                     | Gruppierung von Materialien für Analysen.                                       |
|                            |                                | - `DISPO`: Dispositionskennzeichen            | Kennzeichen für die Steuerung des Materials (z. B. Bestellart).                 |
|                            |                                | - `MAKTX`: Materialkurztext                   | Beschreibender Text zum Material.                                               |
| **Bedarfsdaten**           | BEDZL                         | - `BDMNG`: Bedarfsmengen                     | Zukünftiger Bedarf für jedes Material.                                          |
|                            |                                | - `DATUM`: Bedarfsdatum                      | Datum, zu dem der Bedarf entsteht.                                              |
| **Bestelldaten**           | EKET                          | - `MENGE`: Offene Bestellungen                | Menge, die bestellt, aber noch nicht geliefert wurde.                           |
|                            |                                | - Lieferzeit                                  | Zeitspanne zwischen Bestellung und Lieferung.                                   |
| **Transportdaten**         | LIKP, VTTK, VTTS              | - Lieferscheindaten                           | Details zu transportierten Materialien.                                         |
|                            |                                | - Transportstatus                             | Status des Transports (z. B. geliefert, in Bearbeitung).                         |
| **Kritische Reichweite**   | Berechnet                     | - Material-Reichweite                         | Zeitspanne, in der der Bestand und offene Bestellungen ausreichen, um den Bedarf zu decken. |
|                            |                                | - Engpasswahrscheinlichkeit                   | Wahrscheinlichkeit, dass die Reichweite unter die Mindestanforderung (z. B. Sicherheitsbestand) fällt. |

*Externe Einflussfaktoren* sind nicht verfügbar

### Research question

*Analyse der kritischen Materialreichweiten in:*
. Automatischem Kleinteilelager (AKL)
. Produktions- und Zentrallager (PCC)
. Anlauflagern

*Entwicklung von Regressionsmodellen zur:*
. Vorhersage kritischer Bestandsschwellen
. Minimierung von Versorgungsengpässen
. Optimierung der Lagerhaltung

und so Handlungsempfehlung für die Logistik formulieren zu können. 

### Overview of data

*Beispieldaten:*

In [14]:
import pandas as pd

# Beispieldaten
data = {
    "Materialnummer": ["A0001", "A0002", "A0003", "A0004", "A0005"],
    "Lagerbestand": [150, 300, 50, 200, 100],
    "Offene Bestellungen": [100, 50, 150, 50, 80],
    "Täglicher Bedarf": [20, 30, 10, 25, 15],
    "Lieferzeit (Tage)": [5, 3, 7, 6, 4],
    "Sicherheitsbestand": [100, 200, 80, 150, 90],
    "Bedarfsmengen": [600, 900, 300, 750, 450],
    "Engpasswahrscheinlichkeit (%)": [0.2, 1.0, 0.0, 7.5, 0.7]
}

df = pd.DataFrame(data)

# Datenübersicht
print("Kopf der Daten:")
display(df.head())

print("\nStatistische Zusammenfassung:")
display(df.describe())

Kopf der Daten:


,Materialnummer,Lagerbestand,Offene Bestellungen,Täglicher Bedarf,Lieferzeit (Tage),Sicherheitsbestand,Bedarfsmengen,Engpasswahrscheinlichkeit (%)
0,A0001,150,100,20,5,100,600,0.2
1,A0002,300,50,30,3,200,900,1.0
2,A0003,50,150,10,7,80,300,0.0
3,A0004,200,50,25,6,150,750,7.5
4,A0005,100,80,15,4,90,450,0.7



Statistische Zusammenfassung:


,Lagerbestand,Offene Bestellungen,Täglicher Bedarf,Lieferzeit (Tage),Sicherheitsbestand,Bedarfsmengen,Engpasswahrscheinlichkeit (%)
count,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,160.00000,86.000000,20.000000,5.000000,124.000000,600.000000,1.880000
std,96.17692,41.593269,7.905694,1.581139,50.299105,237.170825,3.166544
min,50.00000,50.000000,10.000000,3.000000,80.000000,300.000000,0.000000
25%,100.00000,50.000000,15.000000,4.000000,90.000000,450.000000,0.200000
50%,150.00000,80.000000,20.000000,5.000000,100.000000,600.000000,0.700000
75%,200.00000,100.000000,25.000000,6.000000,150.000000,750.000000,1.000000
max,300.00000,150.000000,30.000000,7.000000,200.000000,900.000000,7.500000


## Topic idea 2: Häufige Zählaufträge im Logistik-Umfeld mit GenAI - Ursachenanalyse für Anomalien in Zählaufträgen

### Data source

*Der Datensatz stammt aus den internen SAP-basierten Logistiksystemen der Mercedes-Benz AG. Er umfasst Daten zu Zählaufträge, Materialbewegung und Lager- und Materialbestände. Die Daten werden aus verschiedenen SAP-Tabellen wie MARD (Materialbestände), T156 (Bewegungsart), MKPF (Materialbelegkopf) und MSEG (Materialbelegposition).*

Hintergrundinfo: Zähleraufträge werdeb erstellt um Abweichungen im Lagerbestand zu identifizieren. Wenn eine Diskrepanz zwischen dem gebuchten Bestand (im System) und dem physischen Bestand festgestellt wird. Ursachen können fehlerhafte Buchungen, falsch platzierte Materialien oder Diebstahl sein.

### Data characterisitcs

| **Kategorie**              | **Datenquelle (SAP-Tabellen)** | **Merkmale**                                   | **Beschreibung**                                                                                  |
|----------------------------|--------------------------------|-----------------------------------------------|--------------------------------------------------------------------------------------------------|
| **Zählaufträge**           | IKPF, ISEG                    | ZAHL_ID, DATUM, LGORT, WERKS                  | Beschreibt Zählaufträge, deren Zeitpunkt und Lagerorte. Kann manuell oder automatisch erstellt werden. |
| **Materialdaten**          | MAKT, MARD, MSEG             | MATNR, MAKTX, BESTAND, MEINS                  | Informationen über Materialien, deren Beschreibung, verfügbaren Bestand und Maßeinheit.           |
| **Bewegungsdaten**         | MKPF, MSEG                   | BELNR, BUDAT, BWART, MENGE                    | Dokumentiert Materialbewegungen, die den Bestand beeinflussen, z. B. Wareneingänge oder Umbuchungen. |
| **Fehler- und Abweichungen**| QMEL, ISEG                   | ERROR_TYPE, DIFFMENG, ERROR_COMMENTS          | Informationen zu Abweichungen zwischen gezählter und gebuchter Menge sowie protokollierte Fehler. |
| **Prozessdaten**           | IKPF, ISEG                   | USER_ID, COUNT_TIME, SHIFT, METHOD            | Beschreibt Prozessparameter des Zählvorgangs wie Schicht, Zähldauer und die durchführende Person. |

### Research question

*Wie können GenAI-Modelle genutzt werden, um Anomalien in Zählaufträgen zu identifizieren und Handlungsempfehlungen zu generieren?* 

-- Auch hier bin ich mir nicht so ganz sicherob das Thema möglich ist und/oder wie realistisch eine Umsetzung sein kann.

### Overview of data


In [13]:
import pandas as pd

# Datenbeispiel laden
data_zauftraeg = pd.DataFrame({
    "ZAHL_ID": [1001, 1002, 1003],
    "DATUM": ["2024-11-15", "2024-11-16", "2024-11-16"],
    "LGORT": ["L123", "L456", "L123"],
    "MATNR": ["MAT001", "MAT002", "MAT003"],
    "BESTAND": [100, 250, 50],
    "MEINS": ["Stück", "Stück", "Stück"],
    "DIFFMENG": [5, -10, 0],
    "ERROR_TYPE": ["Fehlbuchung", "Überschuss", "Keine Anomalie"],
    "ERROR_COMMENTS": [
        "Material fehlt, Bestand stimmt nicht.",
        "Zu viel gezählt, bitte prüfen.",
        "Keine."
    ],
    "METHOD": ["Manuell", "Autom.", "Manuell"],
    "COUNT_TIME": ["20 min", "10 min", "15 min"]
})

# Datenübersicht
print("Kopf der Daten:")
display(data_zauftraeg.head())

print("\nStatistische Zusammenfassung:")
display(data_zauftraeg.describe())

Kopf der Daten:


,ZAHL_ID,DATUM,LGORT,MATNR,BESTAND,MEINS,DIFFMENG,ERROR_TYPE,ERROR_COMMENTS,METHOD,COUNT_TIME
0,1001,2024-11-15,L123,MAT001,100,Stück,5,Fehlbuchung,"Material fehlt, Bestand stimmt nicht.",Manuell,20 min
1,1002,2024-11-16,L456,MAT002,250,Stück,-10,Überschuss,"Zu viel gezählt, bitte prüfen.",Autom.,10 min
2,1003,2024-11-16,L123,MAT003,50,Stück,0,Keine Anomalie,Keine.,Manuell,15 min



Statistische Zusammenfassung:


,ZAHL_ID,BESTAND,DIFFMENG
count,3.0,3.000000,3.000000
mean,1002.0,133.333333,-1.666667
std,1.0,104.083300,7.637626
min,1001.0,50.000000,-10.000000
25%,1001.5,75.000000,-5.000000
50%,1002.0,100.000000,0.000000
75%,1002.5,175.000000,2.500000
max,1003.0,250.000000,5.000000
